# Series de tiempo.

En esta libreta veremos series de tiempo en pandas. Primero veremos un poco de visualización, manipulación, etc. de series de tiempo. Veremos un poco de teoría "clásica" para lidiar con series de tiempo (muchas veces la teoría clásica le gana a las redes neuronales, sobre todo en series simples). Después, poco a poco, veremos cómo usar redes neuronales para series de tiempo.

Usaremos  `statsmodels` (puedes instalarlo con `conda install statsmodels`).

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt

Vamos a usar un dataset de juguete para experimentar. Contiene las temperaturas mínimas de cada día de 1981 a 1990 en una ciudad de Australia. Lo puedes descargar [aquí](https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv).

In [ ]:
df = pd.read_csv("TimeSeries/daily-min-temperatures.csv"); df

In [ ]:
df = pd.read_csv("TimeSeries/daily-min-temperatures.csv",index_col='Date')

In [ ]:
df

In [ ]:
df.index

In [ ]:
df = pd.read_csv("TimeSeries/daily-min-temperatures.csv",index_col='Date',parse_dates=True)

In [ ]:
df

In [ ]:
df['1985-01-01':'1986-01-01']

In [ ]:
df.describe()

In [ ]:
df.info()

## Visualización en pandas

Siempre, SIEMPRE lo primero que tienes que hacer con un nuevo dataset es VERLO. Entenderlo, ver el formato, qué significan las columnas, etc. etc. etc.

En este caso es súper sencillito, así que con algunas visualizaciones podemos entenderlo.

In [ ]:
df.plot.hist(edgecolor='k',bins=20)

In [ ]:
df.plot.line(y='Temp',figsize=(20,6))

In [ ]:
df.plot.area(alpha=0.5)

In [ ]:
df.plot.box(figsize=(20,6))

In [ ]:
df.plot.kde()

## Agregando columnas.

Digamos que queremos añadir la columna "mes"

In [ ]:
df.index

In [ ]:
df.index.month_name()

In [ ]:
df['Mes'] = df.index.month

In [ ]:
df

In [ ]:
por_mes = df.groupby('Mes').mean()

In [ ]:
por_mes.sort_index()

In [ ]:
df.groupby('Mes').mean().plot.line()

In [ ]:
df["Anio"] = df.index.year

In [ ]:
df

In [ ]:
temp_mensual = df.groupby(['Anio','Mes']).mean()

In [ ]:
temp_mensual[:24].plot.line(figsize=(20,6))

- Para categóricas: bar, barh
- Para ver relaciones entre variables: scatter, con s (size) y c (color), alpha, etc.

### Ejercicio para uds ahorita: 
Analiza 2015-2018-historical-schoo-attendance.csv.

En particular:
- Está separado por escuela. Si quieremos ver el trend "global", ¿cómo lo juntamos? (hint: groupby)
- Crea columnas de porcentajes. Si faltaron 10 de 20 es muy diferente que si faltaron 10 de 10000.
- Visualíza cada variable. 
- Visualiza las parejas de variables. Usa scatter para ver relaciones. hex plots.

## Primera regla de series de tiempo: Valida con **lo último**, no con aleatorios.

In [ ]:
df=df.sort_index()

In [ ]:
len(df)

In [ ]:
train_df = df[:3000]
valid_df = df[3000:]

In [ ]:
train_df

## Baselines

Lo primero que tenemos que hacer es establecer las baselines. Tenemos dos baselines "estándar" para series de tiempo: 
1. Promedio 
2. Repite última. 

A veces funciona mejor una y a veces otra. En el caso de la temperatura: ¿qué creen que funcione mejor? Para discutir: ¿en qué casos creen que funcione mejor una y en qué casos la otra?

Vamos a ver como calcularlas.

### Última

In [ ]:
df

In [ ]:
 df['Temp'].shift(1)

In [ ]:
df['ultima_temp'] = df['Temp'].shift(1)

In [ ]:
df

In [ ]:
df.plot.scatter(x='ultima_temp',y='Temp',alpha=0.5)

In [ ]:
df[['Temp','ultima_temp']].plot.line(figsize=(20,6))

In [ ]:
df[['Temp','ultima_temp']][200:400].plot.line(figsize=(20,6))

In [ ]:
valid_df = df[3000:]

In [ ]:
%%timeit
sum(abs(valid_df['Temp'] - valid_df['ultima_temp']))/len(valid_df)

In [ ]:
%%timeit
error_ultima = np.mean(np.abs(valid_df['Temp'] - valid_df['ultima_temp'])); error_ultima

In [ ]:
def error_l1(df,colA,colB):
    valid_df = df[3000:]
    return np.mean(np.abs(valid_df[colA]-valid_df[colB]))

In [ ]:
def error_rmse(df,colA,colB):
    valid_df = df[3000:]
    return np.sqrt(np.mean(np.square(valid_df[colA]-valid_df[colB])))

In [ ]:
error_rmse(df,"Temp","ultima_temp")

### Promedio

In [ ]:
df['Temp'].expanding()

In [ ]:
df

In [ ]:
df['promedio_temp'] = df['Temp'].expanding().mean().shift(1)

In [ ]:
df

In [ ]:
df[['Temp','promedio_temp']].plot.line(figsize=(20,6))

In [ ]:
error_promedio = error_l1(df,'promedio_temp','Temp'); error_promedio

In [ ]:
error_rmse(df,'promedio_temp','Temp')

Oooh, estuvo mucho peor el error del promedio que de la última. Esto es por las "estaciones".

In [ ]:
df['Temp'][3400:-40].plot()

## Moving average.

En esta misma linea de pensar "baselines" de qué hacer, sigamos con las ideas.

Imaginemos que tenemos una cantidad que va creciendo constantemente poco a poco pero con ruido (e.g. temperatura, economía). Si tomamos el promedio de TODO lo anterior, pues claro que no va a funcionar bien, porque el promedio siempre será más chico que el real. Las temperaturas más recientes son más indicativas de qué está pasando, pero si siempre adivinamos la última, tendríamos mucho ruido.

Realmente, si tú tuvieras que adivinar el siguiente viendo el dibujito, ¿qué harías?

Probablemente se te ocurriría no tomar el anterior, sino el promedio de los $k$   anteriores (con $k=5$ o algún número chiquito).

In [ ]:
df['promedio_ultimas_2'] = df['Temp'].rolling(2).mean().shift(1)
df['promedio_ultimas_3'] = df['Temp'].rolling(3).mean().shift(1)
df['promedio_ultimas_4'] = df['Temp'].rolling(4).mean().shift(1)
df['promedio_ultimas_5'] = df['Temp'].rolling(5).mean().shift(1)
df['promedio_ultimas_50'] = df['Temp'].rolling(50).mean().shift(1)

In [ ]:
df

In [ ]:
[error_l1(df,'Temp',f'promedio_ultimas_{i}') for i in range(2,6)]

In [ ]:
[error_rmse(df,'Temp',f'promedio_ultimas_{i}') for i in range(2,6)]

In [ ]:
df[['Temp','promedio_ultimas_50']].plot.line(figsize=(20,6))

## Seasonality, Trend

En realidad muchos datos tienen una componente de "seasonality", o periodicidad y también una "trend" o "tendencia". Hay miles de maneras de separar en estas dos componentes. Veremos algunas.

### Filtro de Hodrick-Prescott.
La idea es separar la señal en dos componentes así:
    $$y_t = \tau_t + c_t$$
donde $\tau$ denota la tendencia y $c$ la componente cíclica. Los encontramos minimizando lo siguiente:
    $$min_{\tau_t} \sum c_t^2 + \lambda \sum \left[ (\tau_t - \tau_{t-1}) - (\tau_{t-1} -\tau_{t-2}) \right]$$
donde $\lambda$ es un parámetro de suavizado que se puede escoger. Usualmente se pone dependiendo del periodo de repetición (luego veremos otros métodos que lo escogen automáticamente). Por ejemplo, $\lambda = 6.25$ para datos anuales (como el que tenemos).

In [ ]:
import statsmodels as sm
from statsmodels.tsa.filters.hp_filter import hpfilter

In [ ]:
hpfilter??

In [ ]:
temp_trend, temp_cycle  = hpfilter(df['Temp'],lamb=130000)

In [ ]:
df['temp_cycle'] = temp_cycle
df['temp_trend'] = temp_trend

In [ ]:
df[['temp_cycle','temp_trend']].plot()

In [ ]:
df[['Temp','temp_trend']].plot()

Ejercicio: repetir con "macrodata".

In [ ]:
d = pd.read_csv('TimeSeries/macrodata.csv', index_col=0, parse_dates=True)

In [ ]:
d

In [ ]:
d['realgdp'].plot()

In [ ]:
realgdp_cycle, realgdp_trend  = hpfilter(d['realgdp'],lamb=6.25)

In [ ]:
d['realgdp_cycle'] = realgdp_cycle
d['realgdp_trend'] = realgdp_trend

In [ ]:
d[['realgdp_cycle','realgdp_trend']].plot()

## Modelos ETS: Error, Trend, Seasonality



In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
result = seasonal_decompose(df['Temp'],period=365)

In [ ]:
import pylab

In [ ]:
pylab.rcParams['figure.figsize'] = (20.0, 10.0)

In [ ]:
result.plot();

In [ ]:
sb=pd.read_csv('TimeSeries/UDEMY_TSA_FINAL/Data/starbucks.csv')

In [ ]:
result = seasonal_decompose(sb['Close'],period=365)

In [ ]:
result.plot();

## Exponentially weighted moving average

Algo que funciona muy bien muchas veces es calcular el "exponentially weighted moving average": Importan todos los términos anteriores pero cada uno vale un poco menos. Por ejemplo, si los términos de la sucesión son $x_0,x_1,...,x_n$ y nosotros adivinamos $y_0,y_1,\dots,y_n$, podríamos calcular así:
    $$y_n = \frac{\alpha x_{n-1} + \alpha^2 x_{n-2} + \dots + \alpha^n x_0}{\alpha+\alpha^2+\dots+\alpha^n}$$
    
Es decir, estamos haciendo un promedio en donde a cada uno le damos un peso más y más chico que decae exponencialmente.

Otra manera de verlo (mucho más sencilla) es así:
$$y_1 = x_0$$
$$y_n = \alpha x_{n-1} + (1-\alpha) y_{n-1}$$
    
Es decir, para calcular el siguiente término, llevamos "la cuenta" y el siguiente término lo calculamos como $\alpha\ \times$ el nuevo término + $(1-\alpha)\ \times$ lo que ya llevábamos.

Entonces ejercicio: calcúlenlo uds. No tiene que ser eficiente ni nada, solo háganlo.

In [ ]:
def ewma(L,α=0.1):
    valor=L[0]
    resultado=[valor]
    for l in L[1:]:
        valor = α*l+(1-α)*valor
        resultado.append(valor)
    return resultado

In [ ]:
df["temp_ewma"]= ewma(df["Temp"], α = 0.05)

In [ ]:
df

In [ ]:
df[["Temp", "temp_ewma"]].plot()

In [ ]:
df["temp_ewma"]= ewma(df["Temp"], α = 0.8)

In [ ]:
error_l1(df, 'Temp', 'temp_ewma')

In [ ]:
error_rmse(df, 'Temp', 'temp_ewma')

## EWMA en pandas

En pandas ya está EWMA:

In [ ]:
df['EWMA'] = df['Temp'].ewm(alpha=0.3).mean()

In [ ]:
df[200:400].plot(figsize=(20,6))

Hay varias maneras de entender los parámetros de ewm, pero lo que hace es calcular alpha. Por ejemplo, podemos definir la vida media. Lo mejor es jugarle.

## Método de Holt-Winters

¿Cuál es el problema de EWMA? Imaginemos que los valores fueran creciendo constantemente. Sería razonable pensar que seguirá creciendo. EWMA siempre predice valores más pequeños que el último, porque es simplemente un promedio pesado. Vamos entonces a usar una componente de "tendencia" $b_t$, en donde simplemente mediremos qué tanto van creciendo o disminuyendo los valores (método de Holt). Posteriormente le agregaremos una componente de "periodicidad" $s_t$ (seasonality), en donde tomaremos en cuenta los valores del periodo anterior para modelar los ciclos que vemos.

En resumen, usamos 3 componentes: 
1. $\ell_t$ para el "nivel" (EWMA básicamente)
2. $b_t$ para la tendencia
3. $s_t$ para la periodicidad

Cada uno tendrá su parámetro de suavizado: $\alpha, \beta, \gamma$.

Primero veremos una versión más sencilla, el método de Holt, donde solo tomamos en cuenta $\ell$ y $b$.

### Método de Holt
En EWMA teníamos:
$$y_0 = x_0$$
$$y_n = \alpha x_{n} + (1-\alpha) y_{n-1}$$

Para tomar en cuenta la tendencia, debemos medir cómo va cambiando el nivel en cada medición y tomaremos el promedio de éstos:

$$\ell_t = (1-\alpha)\ell_t + \alpha x_t$$
$$b_t = (1-\beta)b_{t-1} + \beta(\ell_t-\ell_{t-1})$$
$$y_t = \ell_t + b_t$$

Prográmalo aquí:

In [ ]:
def holt(X,α,β):
    n = len(X)
    L = [X[0]]
    B = [0]
    Y = [X[0]]
    for i in range(1,n):
        li = α*X[i] + (1-α)*L[i-1]
        L.append(li)
        bi = (1-β)*B[-1] + β*(L[i]-L[i-1])
        B.append(bi)
        yi = li + bi
        Y.append(yi)
    return Y
        

In [ ]:
df['Holt_temp'] = holt(df['Temp'],0.1,0.2)

In [ ]:
df[['Holt_temp','Temp']][200:400].plot()

Ejercicios: 
1. Juega con los parámetros α,β.
2. Pregunta: Si quisiéramos predecir qué pasa en 10 (o 20) tiempos... ¿qué predecimos?

$$ y_{t+k} \approx y_t + k\cdot b_t $$ 

## Método de Holt-Winters: 

Ahora, de la misma manera, podemos entender cómo tomar en cuenta la seasonality. Digamos que creemos que el ciclo debe tener periodo $L$ (e.g. 365). Vamos a considerar un factor $s_t$ multiplicativo (e.g. $y_t = (\ell_t + b_t)s_t$) que irá cambiando con el periodo.

$$\ell_t = (1-\alpha)\ell_t + \alpha x_t$$
$$b_t = (1-\beta)b_{t-1} + \beta(\ell_t-\ell_{t-1})$$
$$s_t = (1-\gamma)s_{t-L} + \gamma(x_t-\ell_{t-1}-b_{t-1}) $$
$$y_t = (\ell_t + b_t)s_t$$

Ejercicios de tarea:
1. Programa este método. Para los primeros $L$ periodos, $s_t = 1$.
2. Grafica $s_t$, $b_t$, $\ell_t$.
3. Pregunta: ¿cómo se hace forecasting con este método? (hint: la respuesta contiene un "módulo L"). 
4. Mide el error.